In [5]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('/home/fran/Escritorio/i3uex/education_drop_clustering/Code/analysis_and_modeling'))
import dunn_index
from sklearn import metrics

%matplotlib inline

In [6]:
analys_personal_data = pd.read_csv('../../../Data/For_analysis_and_modeling/2nd_quadrimester/analys_personal_data.csv',sep='|')

In [7]:
analys_personal_data.head()

,expediente,cod_plan,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,2,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,3,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,4,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,5,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,1,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,6,1623,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,0,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [8]:
interval_cols = []

In [9]:
analys_personal_data_model = analys_personal_data.copy()

In [10]:
analys_personal_data_model.drop(['expediente','cod_plan','abandona'],axis=1,inplace=True)

In [11]:
analys_personal_data_model.head()

,des_plan,anio_apertura_expediente,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.836,D,25,300.6305,S,0.235294,0.176471,1.990064,4.175,5.7,3.84
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,6.836,H,31,667.4115,S,0.312500,0.562500,1.566486,5.000,5.0,2.50
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,6.836,D,28,91.0020,N,0.166667,0.444444,1.345324,3.825,5.2,2.53
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.730,D,25,91.0020,N,0.263158,0.315789,1.850120,3.670,5.4,2.50
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,6.836,H,25,91.0020,N,1.000000,0.000000,1.339003,7.000,7.0,0.00


In [12]:

note_bcket_array = np.array([5, 6.5, 8, 9.5, 10, 11.5, 13, 14])
analys_personal_data_model['nota_admision_def'] = pd.cut(
    analys_personal_data_model['nota_admision_def'], note_bcket_array, include_lowest=True)
interval_cols.append('nota_admision_def')

year_bcket_array = np.array([18, 20, 25, 30, 35, 60])
analys_personal_data_model['edad_acceso'] = pd.cut(
    analys_personal_data_model['edad_acceso'], year_bcket_array, include_lowest=True)
interval_cols.append('edad_acceso')


distance_bcket_array = np.array([0, 45, 90, 135, 250, 500, 1892])
analys_personal_data_model['distance'] = pd.cut(
    analys_personal_data_model['distance'], distance_bcket_array, include_lowest=True)
interval_cols.append('distance')

note_bcket_array = np.array([0, 1.5, 3, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_median'] = pd.cut(
    analys_personal_data_model['cum_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_median')
note_bcket_array = np.array([0, 4.5, 6, 7.5, 9, 10])
analys_personal_data_model['cum_pass_median'] = pd.cut(
    analys_personal_data_model['cum_pass_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_median')
note_bcket_array = np.array([0, 1.5, 3, 5])
analys_personal_data_model['cum_fail_median'] = pd.cut(
    analys_personal_data_model['cum_fail_median'], note_bcket_array, include_lowest=True)
interval_cols.append('cum_fail_median')

ratio_bcket_array = np.array([0, 0.25, 0.5, 0.75, 1])
analys_personal_data_model['cum_pass_ratio'] = pd.cut(
    analys_personal_data_model['cum_pass_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_pass_ratio')

analys_personal_data_model['cum_absent_ratio'] = pd.cut(
    analys_personal_data_model['cum_absent_ratio'], ratio_bcket_array, include_lowest=True)
interval_cols.append('cum_absent_ratio')

std_bcket_array = np.array([0, 0.05,1.15,1.25,2.15,3.25])
analys_personal_data_model['std_deviation'] = pd.cut(
    analys_personal_data_model['std_deviation'], std_bcket_array, include_lowest=True)
interval_cols.append('std_deviation')


In [13]:
def le_dataset(dset, le_cols, cat_cols):
    from sklearn import preprocessing
    for col in cat_cols:
        le = preprocessing.LabelEncoder()
        le.fit(dset[col].cat.categories)
        le_cols.append(le)
        dset[col] = le.transform(dset[col])

In [14]:
def inverse_le_dataset(dset, le_cols, cat_cols):
    from sklearn import preprocessing
    i = 0
    for col in cat_cols:
        le = le_cols[i]
        from sklearn import preprocessing
        dset[col] = le.inverse_transform(dset[col])
        i +=1

In [15]:
def get_dunn_index(data, labels):
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(data)
    return dunn_index.dunn(labels,distances)


In [16]:
analys_personal_data_model.head()

,des_plan,anio_apertura_expediente,convocatoria_acceso,des_acceso,nota_admision_def,sexo,edad_acceso,distance,becario,cum_pass_ratio,cum_absent_ratio,std_deviation,cum_median,cum_pass_median,cum_fail_median
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,"(6.5, 8.0]",D,"(20.0, 25.0]","(250.0, 500.0]",S,"(-0.001, 0.25]","(-0.001, 0.25]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(3.0, 4.5]"
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,"(6.5, 8.0]",H,"(30.0, 35.0]","(500.0, 1892.0]",S,"(0.25, 0.5]","(0.5, 0.75]","(1.25, 2.15]","(4.5, 6.0]","(4.5, 6.0]","(1.5, 3.0]"
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Formación Profesional,"(6.5, 8.0]",D,"(25.0, 30.0]","(90.0, 135.0]",N,"(-0.001, 0.25]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,"(6.5, 8.0]",D,"(20.0, 25.0]","(90.0, 135.0]",N,"(0.25, 0.5]","(0.25, 0.5]","(1.25, 2.15]","(3.0, 4.5]","(4.5, 6.0]","(1.5, 3.0]"
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,JUN,Selectividad,"(6.5, 8.0]",H,"(20.0, 25.0]","(90.0, 135.0]",N,"(0.75, 1.0]","(-0.001, 0.25]","(1.25, 2.15]","(6.0, 7.5]","(6.0, 7.5]","(-0.001, 1.5]"


In [17]:
analys_personal_data_model = pd.get_dummies(analys_personal_data_model)

In [18]:
analys_personal_data_model.head()

,des_plan_GRADO EN EDIFICACIÓN,des_plan_GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIVILES,des_plan_GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA,des_plan_GRADO EN INGENIERÍA CIVIL - TRANSPORTES Y SERVICIOS URBANOS,des_plan_GRADO EN INGENIERÍA DE SONIDO E IMAGEN EN TELECOMUNICACIÓN,des_plan_GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DE COMPUTADORES,des_plan_GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DEL SOFTWARE,anio_apertura_expediente_2008-09,anio_apertura_expediente_2009-10,anio_apertura_expediente_2010-11,...,"cum_pass_median_(6.0, 7.5]","cum_pass_median_(7.5, 9.0]","cum_pass_median_(9.0, 10.0]","cum_fail_median_(-0.001, 1.5]","cum_fail_median_(1.5, 3.0]","cum_fail_median_(3.0, 4.5]","cum_fail_median_(4.5, 6.0]","cum_fail_median_(6.0, 7.5]","cum_fail_median_(7.5, 9.0]","cum_fail_median_(9.0, 10.0]"
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0


In [19]:
from sklearn.preprocessing import MinMaxScaler

norm = MinMaxScaler().fit(analys_personal_data_model)
analys_personal_data_model[analys_personal_data_model.columns] = norm.transform(analys_personal_data_model)

In [31]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.90, svd_solver="auto")
mnist_pca = pca.fit(analys_personal_data_model)
analys_personal_model_pca = mnist_pca.transform(analys_personal_data_model)
analys_personal_model_pca = pd.DataFrame(data=analys_personal_model_pca)

pca.n_components_


31

In [32]:
from sklearn.cluster import AgglomerativeClustering
aglom_cluster = AgglomerativeClustering(n_clusters = 2, affinity='euclidean', linkage='ward')
aglom_cluster.fit_predict(analys_personal_model_pca)
analys_personal_data_clust = analys_personal_model_pca.copy()
analys_personal_data_clust['labels'] = aglom_cluster.labels_
analys_personal_data['labels'] = analys_personal_data_clust['labels'].astype('category')

/home/franjmelchor/.virtualenvs/education_drop_clustering/lib/python3.10/site-packages/sklearn/cluster/_agglomerative.py:1005: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [33]:
metrics.silhouette_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


0.09675253980323636

In [34]:
metrics.calinski_harabasz_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


193.39707965642768

In [35]:
metrics.davies_bouldin_score(analys_personal_data_clust, analys_personal_data_clust['labels'])


2.9397267024807014

In [36]:
get_dunn_index(analys_personal_data_clust,analys_personal_data_clust['labels'])

0.3149480780512655

In [37]:
import pandas as pd
def get_impurity(dropout_cl0: pd.core.series.Series, dropout_cl1:pd.core.series.Series):
    impurity = -1
    n = len (dropout_cl0) + len (dropout_cl1)
    if len(dropout_cl0[dropout_cl0 == 0]) > len(dropout_cl0[dropout_cl0 == 1]):
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 0]))/n
    else:
        impurity = (len(dropout_cl0) - len(dropout_cl0[dropout_cl0 == 1]))/n
    if len(dropout_cl1[dropout_cl1 == 0]) > len(dropout_cl1[dropout_cl1 == 1]):
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 0]))/n)
    else:
        impurity = impurity + ((len(dropout_cl1) - len(dropout_cl1[dropout_cl1 == 1]))/n)
        
    return impurity

In [38]:
analys_personal_data_clust['abandona'] = analys_personal_data['abandona']

In [39]:
get_impurity(analys_personal_data_clust[analys_personal_data_clust['labels']==0]['abandona'], analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'])

0.25888324873096447

In [40]:
analys_personal_data_clust[analys_personal_data_clust['labels']==0]['abandona'].value_counts(normalize=True)

1    0.720339
0    0.279661
Name: abandona, dtype: float64

In [41]:
analys_personal_data_clust[analys_personal_data_clust['labels']==1]['abandona'].value_counts(normalize=True)

0    0.772152
1    0.227848
Name: abandona, dtype: float64

In [28]:
# from apitep_utils.report import Report
# for label in analys_personal_data['labels'].cat.categories:
#     dset = analys_personal_data[analys_personal_data['labels'] == label]
#     report = Report()
#     report.generate_advanced(dset,'KMeans_1st_year_2_cl_'+str(label),sys.path[0]+ '/'+ '1st_year_cat_'+ str(label))